In [1]:
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import scipy.io, cv2

# Find images path
scene_paths = glob('../Dataset_CNN/*/')

# Fulfill data
X_ = []
y_ = []
for scene in scene_paths[0:3]:
    print(scene)
    windows_path = glob(scene + '*')
    
    for window in windows_path:
        images_ref_path = glob(window + '/w_*.jpg')
        NLF_path = window + '/NLF.mat'
        
        mat = scipy.io.loadmat(NLF_path)
        NLF = mat["NLF_step"][0]
        
        for img_ref in images_ref_path:
            img = plt.imread(img_ref)
            
            X_.append(img)
            y_.append(np.nan_to_num(NLF))
            
        

In [2]:
X_ = np.asarray(X_)
y_ = np.asarray(y_)


In [3]:
print(np.shape(X_))
print(np.shape(y_))

(22680, 256, 256, 3)
(22680, 192)


### Crear modelo

In [8]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, Reshape
from keras.layers import concatenate, AveragePooling2D, Activation, BatchNormalization
from keras.models import Model
from SpatialPyramidPooling import SpatialPyramidPooling
import keras
# First, define the vision modules
input_imgs = Input(shape=(256, 256, 3))

# Model
def get_model_column(ks_1, ks_2,p_):
    x = Conv2D(64, kernel_size=(ks_1,ks_1), strides = 2, activation=None)(input_imgs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D((2, 2))(x)
    x = Conv2D(64, kernel_size=(ks_2,ks_2), activation=None)(x) # conv2_a
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D((2, 2))(x) # pool2_a
    x = Conv2D(64, kernel_size=(ks_2,ks_2), activation=None)(x) # conv2_a
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, kernel_size=(ks_2,ks_2), activation=None)(x) # conv2_a
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #out = keras.layers.ZeroPadding2D(padding=((p_, 0), (0, p_)), data_format=None)(x)
    out = Flatten()(x)
    return out

vision_model1 = Model(input_imgs, get_model_column(5, 3, 0))
vision_model2 = Model(input_imgs, get_model_column(7, 5, 5))
vision_model3 = Model(input_imgs, get_model_column(9, 7, 10))

# Then define the input-apart model
input_a = Input(shape=(256, 256, 3))
input_b = Input(shape=(256, 256, 3)) #shape=(300, 300, 3))
input_c = Input(shape=(256, 256, 3)) #shape=(340, 340, 3))

# The vision model will be shared, weights and all
out_a = vision_model1(input_a)
out_b = vision_model2(input_b)
out_c = vision_model3(input_c)

concatenated = concatenate([out_a, out_b, out_c],axis=1)
#out = Dense(1, activation='sigmoid')(concatenated)
#merge_abc = SpatialPyramidPooling([1, 2, 4])(concatenated)
merge_abc = Dense(2048, activation='relu')(concatenated)
#merge_abc = Dense(256*3, activation='relu')(merge_abc)
merge_abc = Dense(64*3, activation='relu')(merge_abc)

model_abc = Model([input_a, input_b, input_c], merge_abc)

In [9]:
model_abc.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
model_8 (Model)                 (None, 43264)        116672      input_11[0][0]                   
__________________________________________________________________________________________________
model_9 (M

### Training

In [10]:
from keras import optimizers

sgd = optimizers.SGD(lr=0.001, decay=0.0001, momentum=0.9, nesterov=True)

# compile model (abc)
model_abc.compile(optimizer= sgd, loss='mse')

In [11]:
# Make model_a non trainable
model_abc.layers[3].trainable = False
model_abc.layers[4].trainable = False
model_abc.fit(x=[X_,X_,X_], y=y_, batch_size=32, epochs=10, validation_split=0.1, shuffle=True)

# Make model_a non trainable
model_abc.layers[3].trainable = False
model_abc.layers[4].trainable = True
model_abc.layers[5].trainable = False
model_abc.fit(x=[X_,X_,X_], y=y_, batch_size=32, epochs=10, validation_split=0.1, shuffle=True)

# Make model_a non trainable
model_abc.layers[3].trainable = True
model_abc.layers[4].trainable = False
model_abc.layers[5].trainable = False
model_abc.fit(x=[X_,X_,X_], y=y_, batch_size=32, epochs=10, validation_split=0.1, shuffle=True)


Instructions for updating:
Use tf.cast instead.


/Users/Carlusaurio/.pyenv/versions/3.6.6/envs/venv_tfg/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 20412 samples, validate on 2268 samples
Epoch 1/10
   32/20412 [..............................] - ETA: 6:42:52 - loss: nan

KeyboardInterrupt: 

In [ ]:

(model_abc.predict([[X_[0]],[X_[0]],[X_[0]]]))

In [ ]:
import matplotlib.pyplot as plt
y_pred = np.transpose(model_abc.predict([[X_[0]],[X_[0]],[X_[0]]]))

fig, ax = plt.subplots()
ax.plot(y_pred[0:256],'r')

ax.set(xlabel='intensity', ylabel='std',
       title='pred nlf')
plt.show()

fig, ax = plt.subplots()
ax.plot(y_[0][0:256],'r')

ax.set(xlabel='intensity', ylabel='std',
       title='test nlf')
plt.show()

In [ ]:
from keras.utils import plot_model

plot_model(model_abc, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
import numpy as np
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
import scipy.io


scene = 'Scene_21'
nlf_path = '../Dataset_Final/NLFs/NLF_' + scene + '.mat'
mat = scipy.io.loadmat('../Dataset_Final/NLFs/NLF_' + scene + '.mat')
NLF = mat["NLF"][0]
NLF_r = np.nan_to_num(NLF[0:256])

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y,b,a

order = 5
fs = 30    # sample rate, Hz
cutoff = 2
y,b,a = butter_lowpass_filter(NLF_r, cutoff, fs, order)

# Plot the frequency response.
w, h = freqz(b, a, worN=8000)

plt.plot(0.5*fs*w/np.pi, np.abs(h), 'b')
plt.plot(cutoff, 0.5*np.sqrt(2), 'ko')
plt.axvline(cutoff, color='k')
plt.xlim(0, 0.5*fs)
plt.title("Lowpass Filter Frequency Response")
plt.xlabel('Frequency [Hz]')
plt.grid()

fig, ax = plt.subplots()
ax.plot(y,'r')

ax.set(xlabel='intensity', ylabel='std',
       title='test nlf filtered')
plt.show()

fig, ax = plt.subplots()
ax.plot(NLF_r,'r')

ax.set(xlabel='intensity', ylabel='std',
       title='test nlf')
plt.show()